# Redis Bike Co with polars-redis

This notebook demonstrates how to use **polars-redis** to work with the Redis Bike Co dataset - a fictional bicycle retail company with 111 bikes and 5 stores.

We'll cover:
1. Loading data into Redis
2. Schema inference
3. Scanning and querying data
4. Using RediSearch for filtering
5. Aggregations
6. Comparison with traditional redis-py approach

## Setup

First, make sure you have Redis Stack running:

```bash
docker run -d --name redis-stack -p 6379:6379 redis/redis-stack:latest
```

In [ ]:
import json
from pathlib import Path

import polars as pl
import polars_redis as redis

# Redis connection URL
REDIS_URL = "redis://localhost:6379"

# Data directory
DATA_DIR = Path("data")

## 1. Loading Data into Redis

Let's load the bike and store data from JSON files into Redis using `write_json()`.

In [ ]:
# Load bikes from JSON
with open(DATA_DIR / "bikes.json") as f:
    bikes_raw = json.load(f)["data"]

# Flatten the nested structure into a DataFrame
bikes_df = pl.DataFrame([
    {
        "stockcode": b["stockcode"],
        "model": b["model"],
        "brand": b["brand"],
        "price": b["price"],
        "type": b["type"],
        "description": b["description"],
        "material": b["specs"]["material"],
        "weight": b["specs"]["weight"],
    }
    for b in bikes_raw
])

print(f"Loaded {len(bikes_df)} bikes")
bikes_df.head()

In [ ]:
# Write bikes to Redis as JSON documents
result = redis.write_json(
    bikes_df,
    url=REDIS_URL,
    key_column="stockcode",
    key_prefix="redisbikeco:bike:",
)

print(f"Wrote {result.success_count} bikes to Redis")

In [ ]:
# Load and write stores
with open(DATA_DIR / "stores.json") as f:
    stores_raw = json.load(f)["data"]

stores_df = pl.DataFrame([
    {
        "storecode": s["storecode"],
        "storename": s["storename"],
        "city": s["address"]["city"],
        "state": s["address"]["state"],
        "position": s["position"],
        "amenities": ",".join(s["amenities"]),
    }
    for s in stores_raw
])

result = redis.write_json(
    stores_df,
    url=REDIS_URL,
    key_column="storecode",
    key_prefix="redisbikeco:store:",
)

print(f"Wrote {result.success_count} stores to Redis")
stores_df

## 2. Schema Inference

polars-redis can automatically infer the schema from your Redis data.

In [ ]:
# Infer schema from bike documents
inferred_schema = redis.infer_json_schema(
    REDIS_URL,
    pattern="redisbikeco:bike:*",
    sample_size=10,
)

print("Inferred bike schema:")
for field, dtype in inferred_schema.items():
    print(f"  {field}: {dtype}")

In [ ]:
# Get schema with confidence scores
schema_with_confidence = redis.infer_json_schema_with_confidence(
    REDIS_URL,
    pattern="redisbikeco:bike:*",
    sample_size=20,
)

print("Schema inference confidence:")
for field, info in schema_with_confidence.fields.items():
    print(f"  {field}: {info.inferred_type} (confidence: {info.confidence:.1%})")

## 3. Scanning Data

Use `scan_json()` to lazily read all documents matching a pattern.

In [ ]:
# Define the schema for bikes
bike_schema = {
    "stockcode": pl.Utf8,
    "model": pl.Utf8,
    "brand": pl.Utf8,
    "price": pl.Int64,
    "type": pl.Utf8,
    "description": pl.Utf8,
    "material": pl.Utf8,
    "weight": pl.Float64,
}

# Create a lazy scan
bikes_lf = redis.scan_json(
    REDIS_URL,
    pattern="redisbikeco:bike:*",
    schema=bike_schema,
)

# Nothing has been fetched yet - this is lazy!
print(f"LazyFrame schema: {bikes_lf.schema}")

In [ ]:
# Collect to execute the scan
all_bikes = bikes_lf.collect()

print(f"Total bikes: {len(all_bikes)}")
all_bikes.head(10)

## 4. Polars Operations

Once data is in a DataFrame, you have full Polars power!

In [ ]:
# Find bikes by brand
ergonom_bikes = all_bikes.filter(pl.col("brand") == "Ergonom")
print(f"Ergonom bikes: {len(ergonom_bikes)}")
ergonom_bikes

In [ ]:
# Price statistics by type
price_stats = (
    all_bikes
    .group_by("type")
    .agg(
        pl.col("price").count().alias("count"),
        pl.col("price").mean().alias("avg_price"),
        pl.col("price").min().alias("min_price"),
        pl.col("price").max().alias("max_price"),
    )
    .sort("avg_price", descending=True)
)

# Note: Prices are in cents (paise), convert to rupees for display
price_stats.with_columns(
    (pl.col("avg_price") / 100).alias("avg_price_rs"),
    (pl.col("min_price") / 100).alias("min_price_rs"),
    (pl.col("max_price") / 100).alias("max_price_rs"),
)

In [ ]:
# Weight analysis by material
weight_by_material = (
    all_bikes
    .group_by("material")
    .agg(
        pl.col("weight").count().alias("count"),
        pl.col("weight").mean().alias("avg_weight_kg"),
        pl.col("weight").min().alias("min_weight_kg"),
        pl.col("weight").max().alias("max_weight_kg"),
    )
    .sort("avg_weight_kg")
)

weight_by_material

In [ ]:
# Find budget-friendly bikes (under 50000 cents = 500 Rs)
budget_bikes = (
    all_bikes
    .filter(pl.col("price") < 50000)
    .sort("price")
    .select(["stockcode", "brand", "model", "type", "price"])
)

print(f"Budget bikes (under Rs 500): {len(budget_bikes)}")
budget_bikes

In [ ]:
# Lightweight carbon bikes
lightweight_carbon = (
    all_bikes
    .filter(
        (pl.col("material").str.contains("carbon")) &
        (pl.col("weight") < 12)
    )
    .sort("weight")
    .select(["brand", "model", "type", "material", "weight", "price"])
)

print(f"Lightweight carbon bikes (<12kg): {len(lightweight_carbon)}")
lightweight_carbon

## 5. Creating a RediSearch Index

To use `search_json()`, we need a RediSearch index. Let's create one using redis-py.

In [ ]:
import redis as redis_py

r = redis_py.from_url(REDIS_URL)

# Drop existing index if present
try:
    r.execute_command("FT.DROPINDEX", "bikes_idx", "DD")
    print("Dropped existing index")
except:
    pass

# Create index
r.execute_command(
    "FT.CREATE", "bikes_idx",
    "ON", "JSON",
    "PREFIX", "1", "redisbikeco:bike:",
    "SCHEMA",
    "$.stockcode", "AS", "stockcode", "TAG",
    "$.model", "AS", "model", "TEXT",
    "$.brand", "AS", "brand", "TAG",
    "$.price", "AS", "price", "NUMERIC", "SORTABLE",
    "$.type", "AS", "type", "TAG",
    "$.description", "AS", "description", "TEXT",
    "$.material", "AS", "material", "TAG",
    "$.weight", "AS", "weight", "NUMERIC", "SORTABLE",
)

print("Created bikes_idx index")

## 6. Using RediSearch with Query Builder

Now we can use `search_json()` with the polars-redis query builder for efficient server-side filtering.

In [ ]:
from polars_redis.query import col

# Search for eBikes
ebikes = redis.search_json(
    REDIS_URL,
    index="bikes_idx",
    query=col("type") == "eBikes",
    schema=bike_schema,
).collect()

print(f"eBikes found: {len(ebikes)}")
ebikes.select(["stockcode", "brand", "model", "price"])

In [ ]:
# Complex query: Mountain bikes under 200000 with carbon frame
query = (
    (col("type") == "Mountain Bikes") &
    (col("price") < 200000) &
    (col("material") == "carbon")
)

print(f"Query: {query.to_redis()}")

results = redis.search_json(
    REDIS_URL,
    index="bikes_idx",
    query=query,
    schema=bike_schema,
).collect()

print(f"\nFound {len(results)} bikes")
results.select(["brand", "model", "price", "material", "weight"])

In [ ]:
# Text search in descriptions
trail_bikes = redis.search_json(
    REDIS_URL,
    index="bikes_idx",
    query=col("description").contains("trail"),
    schema=bike_schema,
).collect()

print(f"Bikes with 'trail' in description: {len(trail_bikes)}")
trail_bikes.select(["brand", "model", "type", "description"]).head(5)

In [ ]:
# Price range query
mid_range = redis.search_json(
    REDIS_URL,
    index="bikes_idx",
    query=col("price").is_between(100000, 200000),
    schema=bike_schema,
).collect()

print(f"Mid-range bikes (Rs 1000-2000): {len(mid_range)}")
mid_range.select(["brand", "model", "type", "price"]).sort("price").head(10)

## 7. Aggregations

Use `aggregate_json()` for server-side aggregations with RediSearch.

In [ ]:
# Count bikes by type using RediSearch aggregation
from polars_redis._search import aggregate_json

type_counts = aggregate_json(
    REDIS_URL,
    index="bikes_idx",
    query="*",
    group_by=["type"],
    reducers=["COUNT"],
).collect()

type_counts.sort("__generated_aliascount", descending=True)

In [ ]:
# Average price by brand
brand_avg = aggregate_json(
    REDIS_URL,
    index="bikes_idx",
    query="*",
    group_by=["brand"],
    reducers=["COUNT", "AVG(price)"],
).collect()

brand_avg.sort("__generated_aliasavg_price", descending=True).head(10)

## 8. Comparison: Traditional vs polars-redis

Let's compare the traditional redis-py approach with polars-redis.

In [ ]:
# Traditional approach with redis-py
import redis as redis_py
import json

r = redis_py.from_url(REDIS_URL)

# Get all bike keys
keys = list(r.scan_iter("redisbikeco:bike:*"))

# Fetch each document
bikes_traditional = []
for key in keys:
    data = r.json().get(key)
    if data:
        bikes_traditional.append(data)

# Convert to DataFrame manually
df_traditional = pl.DataFrame(bikes_traditional)

print(f"Traditional approach: {len(df_traditional)} bikes")
df_traditional.head(3)

In [ ]:
# polars-redis approach - much simpler!
df_polars_redis = redis.scan_json(
    REDIS_URL,
    pattern="redisbikeco:bike:*",
    schema=bike_schema,
).collect()

print(f"polars-redis approach: {len(df_polars_redis)} bikes")
df_polars_redis.head(3)

## Summary

With **polars-redis**, working with Redis data becomes much simpler:

1. **Write data** with `write_json()` - just pass a DataFrame
2. **Infer schema** with `infer_json_schema()` - no manual type mapping
3. **Scan data** with `scan_json()` - lazy evaluation, batched fetching
4. **Search data** with `search_json()` - leverage RediSearch with a Pythonic query builder
5. **Aggregate data** with `aggregate_json()` - server-side aggregations

The result is less boilerplate, better performance through batching, and the full power of Polars for data analysis.